In [4]:
import pandas as pd
from numpy.linalg import inv
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import multivariate_normal

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.pipeline import make_pipeline

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
def get_pi_hat(pd_serie, z):
    # get the proportion of class z
    return sum(pd_serie == z)/len(pd_serie)


def Gaussian_empirical_parameters(X, Y, z1=1, z2=2):
    # get gaussian empirical parameters from data
    pi_hat1 = get_pi_hat(Y, z1) # portion of class1
    pi_hat2 = get_pi_hat(Y, z2) # portion of class2
    X_1 = X[Y==z1]
    X_2 = X[Y==z2]
    mu1 = X_1.mean(axis=0)
    mu2 = X_2.mean(axis=0)
    n_1 = len(X_1)
    n_2 = len(X_2)

    x1_sub_mu1=np.array(X_1-mu1)
    x2_sub_mu2=np.array(X_2-mu2)

    V_1 = (x1_sub_mu1.T@x1_sub_mu1)/n_1
    V_2 = (x2_sub_mu2.T@x2_sub_mu2)/n_2
    return pi_hat1, pi_hat2, n_1, n_2, mu1, mu2, V_1, V_2


def is_pos_def(x):
    # verify if x is positive definite
    return np.all(np.linalg.eigvals(x) > 0)


def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

In [69]:
data = pd.read_csv('data/bcw.csv')
Y = data.z.to_numpy()
X = data.drop(columns=["z"]).to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3)
theta1, theta2, n1, n2, mu1, mu2, cov_1, cov_2 = Gaussian_empirical_parameters(X_train, y_train)

In [71]:
print(is_pos_def(cov_1))
print(is_pos_def(cov_2))
print(check_symmetric(cov_1))
print(check_symmetric(cov_2))

True
True
True
True


## Regularized discriminant analysis

In [ ]:
def regularized_Gaussian_DA(X, Y, mu, kappa, Lambda, z1=1, z2=2, nu=None):
    ''' Gaussian DA with Gaussian inverse-Wishart prior
        the maximum-likelihood estimated mu and sigma by posterior distribution
        We suppose these two class have the same prior distribution for the parameters
    '''
    _, _, n_1, n_2, Xm_1, Xm_2, V_1, V_2 = Gaussian_empirical_parameters(X, Y, z1=z1, z2=z2)

    mu_hat_1 = (n_1 * Xm_1 + kappa * mu)/(n1 + kappa)
    mu_hat_2 = (n_2 * Xm_2 + kappa * mu)/(n2 + kappa)
    
    d = X.shape[1]
    if nu is None:
        nu = X.shape[1] + 1
    m_mu_1 = (Xm_1 - mu).reshape(-1,1)
    m_mu_2 = (Xm_2 - mu).reshape(-1,1)
    sigma_hat_1 = (n_1 * V_1 + ((n1 * kappa)/(n1 + kappa)) * np.dot(m_mu_1,m_mu_1.T) + inv(Lambda))/(n_1 + nu + d + 2)
    sigma_hat_2 = (n_2 * V_2 + ((n2 * kappa)/(n2 + kappa)) * np.dot(m_mu_2,m_mu_2.T) + inv(Lambda))/(n_2 + nu + d + 2)
    return mu_hat_1, mu_hat_2, sigma_hat_1, sigma_hat_2

def RGDA_prediction(X_train, y_train, X_test, y_test, mu, kappa, Lambda, z1=1, z2=2, nu=None):
    mu_hat_1, mu_hat_2, sigma_hat_1, sigma_hat_2 = regularized_Gaussian_DA(X_train, y_train, mu, kappa, Lambda, z1, z2, nu)
    p1 = multivariate_normal.pdf(X_test, mean=mu_hat_1, cov=sigma_hat_1)
    p2 = multivariate_normal.pdf(X_test, mean=mu_hat_2, cov=sigma_hat_2)
    y_pred = (p1 > p2) * -1 + 2
    print(f'accuracy:\n {accuracy_score(y_test, y_pred)}')
    print(f'confusion_matrix:\n {confusion_matrix(y_test, y_pred)}')

def run_RGDA_prediction(data_path):
    data = pd.read_csv(data_path)
    y = data.z.to_numpy()
    X = data.drop(columns=["z"]).to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=30)
    d = X.shape[1]
    mu = 0
    kappa = 1
    Lambda = np.eye(d)
    RGDA_prediction(X_train, y_train, X_test, y_test, mu, kappa, Lambda)

In [103]:
data = pd.read_csv('data/bcw.csv')
y = data.z.to_numpy()
X = data.drop(columns=["z"]).to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
d = X.shape[1]
mu = 0
kappa = 1
Lambda = np.eye(d)
prediction(X_train, y_train, X_test, y_test, mu, kappa, Lambda)

accuracy:
 0.9512195121951219
confusion_matrix:
 [[127   8]
 [  2  68]]


## Compare the results with those obtained with regulized quadratic logistic regression

In [ ]:
def quadratic_LR(X_train, y_train, X_test, y_test)
poly = PolynomialFeatures(degree=2, include_bias=False)
cls_skl = SklearnLogisticRegression(max_iter=1000, solver='newton-cg', penalty='none')
pipe_skl = make_pipeline(poly, cls_skl)
pipe_skl.fit(X, y)

In [ ]:

#fig, axs = plt.subplots(1, 3, tight_layout=True)
#sns.scatterplot(x="X1", y="X2", hue="z", data=Xy, ax=axs[0]) add_decision_boundary(pipe_skl, label="SkPolyLR", ax=axs[0])





In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
cls = LogisticRegression()
pipe = make_pipeline(poly, cls)
pipe.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('polynomialfeatures',
                 PolynomialFeatures(degree=2, include_bias=False,
                                    interaction_only=False, order='C')),
                ('logisticregression',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [ ]:
pipe.predict(x_test)

array([1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1,
       2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1,
       1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1,
       1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1,
       1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2,
       2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1,
       1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1])